<a href="https://colab.research.google.com/github/akaanurag281/TheHinduFeedParser/blob/main/GithubPythonTheHinduFeedParse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np

Mounted at /content/drive


In [ ]:
!pip install feedparser
!pip install requests
!pip install beautifulsoup4
!pip install openai -q

In [ ]:
from openai import OpenAI
import os
os.environ['OPENAI_API_KEY'] = "YOUR_KEY"
client = OpenAI()
def generate_summarizer(
    prompt
):
    res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens=256,
        temperature=0.1,
        top_p=0.1,
        frequency_penalty=1,
        messages=
       [
         {
          "role": "system",
          "content": "You are a helpful assistant for text summarization.",
         },
         {
          "role": "user",
          "content": f"Summarize this {prompt}",
         },
        ],
    )
    return res.choices[0].message.content
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   response = client.embeddings.create(input=[text], model=model)
   return response.data[0].embedding
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
def get_news_content(url,is_ai):
  response = requests.get(url)

  if response.status_code == 200:
    news_content = response.text
    soup = BeautifulSoup(news_content, 'html.parser')
    content_element = soup.find('div', {'class': 'articlebodycontent', 'itemprop': 'articleBody'})
    if content_element:
      news_content = content_element.get_text()
      if is_ai:
        return generate_summarizer(extract_text_before_comments(news_content))
      else:
        return extract_text_before_comments(news_content)
    else:
      print("Content element not found")
  else:
    print("Failed to fetch news content from the provided link")

def extract_text_before_comments(text):
    # Define a pattern to find "Comments" in a contextually irrelevant position (case-insensitive)
    pattern = re.compile(r"comments", re.IGNORECASE)
    # Find the index where "Comments" appears in the text
    match = re.search(pattern, text)
    if match:
        # Extract text before the occurrence of "Comments"
        filtered_text = text[:match.start()].strip()
        return filtered_text
    else:
        return "No 'Comments' found or improperly formatted text"

In [ ]:
df = pd.read_csv('PATH_OF_PYQ_WITH_EMBEDDING')
df['embedding'] = df['embedding'].apply(eval).apply(np.array ) #to be used for Numpy calculation
#df Show output

In [ ]:
import feedparser
import os
from datetime import datetime, timedelta

def fetch_news_from_feed(rss_url):
    # Initialize empty lists to store data
    titles = []
    links = []
    summaries = []
    news_contents = []
    today = datetime.now().date()
    yesterday = today - timedelta(days=1)

    # Parse the RSS feed
    feed = feedparser.parse(rss_url)
    # Extract and print news content
    for entry in feed.entries:
        published_date = datetime.strptime(entry.published, "%a, %d %b %Y %H:%M:%S %z").date()
        #if published_date == yesterday or published_date == today:
        if  published_date == today:
            titles.append(entry.title)
            links.append(entry.link)
            summaries.append(entry.summary)
            news_contents.append(get_news_content(entry.link, is_ai=False))
    data = {
    'Title': titles,
    'Link': links,
    'Summary': summaries,
    'News Content': news_contents
        }
    return pd.DataFrame(data)


all_news_df = pd.DataFrame()
# List of RSS feed URLs
rss_feeds = [
    'https://www.thehindu.com/sci-tech/science/feeder/default.rss',
    'https://www.thehindu.com/business/agri-business/feeder/default.rss',
    'https://www.thehindu.com/business/Industry/feeder/default.rss',
    'https://www.thehindu.com/business/Economy/feeder/default.rss',
    'https://www.thehindu.com/business/budget/feeder/default.rss',
    'https://www.thehindu.com/news/national/feeder/default.rss',
     'https://www.thehindu.com/news/international/feeder/default.rss',
    '#https://www.thehindu.com/news/states/feeder/default.rss',
    'https://www.thehindu.com/news/cities/feeder/default.rss',
    'https://www.thehindu.com/opinion/columns/feeder/default.rss',
    'https://www.thehindu.com/opinion/editorial/feeder/default.rss'
    # Add more feed URLs here...
]

csv_file_path = '/content/sample_data/todays_epaper.csv'
# Check if the file exists
if not os.path.exists(csv_file_path):
  # Fetch news from each RSS feed
     for rss_url in rss_feeds:
         print(f"Fetching news from {rss_url}")
         today_news_df = fetch_news_from_feed(rss_url)
         all_news_df = pd.concat([all_news_df, today_news_df], ignore_index=True)

     all_news_df['Combined_Text'] = all_news_df['Title'] + '-' + all_news_df['Summary'].astype(str)
     all_news_df['embedding'] = all_news_df['Combined_Text'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
     all_news_df.to_csv(csv_file_path)
     print("File saved successfully.")
else:
    print("File already exists. Skipping the read operation.")
    all_news_df = pd.read_csv(csv_file_path)

all_news_df['embedding'] = all_news_df['embedding'].apply(eval).apply(np.array )
#print(all_news_df)


File already exists. Skipping the read operation.


In [ ]:
#finding news relevant for syllabus and PYQ
final_news = []
for index, line in all_news_df.iterrows():
     # Adjust cosine similarity 0.830 as per use case
     filtered_df = df[df['embedding'].apply(lambda x: cosine_similarity(x, line['embedding']) > 0.830)]
     if not filtered_df.empty:
       final_news.append(line['Link'])

final_news_df = pd.DataFrame(final_news, columns=['Link'])
final_news_df.to_csv('/content/sample_data/relevant_news_link.csv')